In [12]:
!python --version

Python 3.7.7


## 加载数据到内存

In [13]:
import librosa
import numpy as np
import os
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
data_set = []
label_set = []
label2id = {genre:i for i,genre in enumerate(genres)}
id2label = {i:genre for i,genre in enumerate(genres)}
print(label2id)
for g in genres:
    print(g)
    for filename in os.listdir(f'../dataset/genres/{g}/'):
        songname = f'../dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        data_set.append([float(i) for i in to_append.split(" ")])
        label_set.append(label2id[g])

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
blues
classical
country
disco
hiphop
jazz
metal
pop
reggae
rock


## 创建数据集

In [14]:
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data_set, dtype = float))
y = np_utils.to_categorical(np.array(label_set))

Using TensorFlow backend.


In [15]:
print("X.shape: ", X.shape, " Y.shape:", y.shape)

X.shape:  (1000, 26)  Y.shape: (1000, 10)


### 将测试集和训练集分隔

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 创建模型

In [17]:
from keras import models
from keras.layers import Dense, Dropout
def create_model():
    model = models.Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model
model = create_model()

这里创建了一个包含三个隐藏层的神经网络，最后一层输出的是分类层，因为是10类，所以最后一层是10个单元。（这里增加了一层Dropout减少数据过拟合）

## 编译模型

这里是一个分类问题，所以使用类别交叉熵函数`categorical_crossentropy`，然后优化器选择`Adam`，评价指标选择正确率。

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 训练与评估

接下来使用`fit`方法进行训练，训练50轮。

In [19]:
model.fit(X_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
800/800 [==============================] - 1s 690us/step - loss: 2.2582 - accuracy: 0.1813
Epoch 2/50
800/800 [==============================] - 0s 58us/step - loss: 2.0312 - accuracy: 0.3150
Epoch 3/50
800/800 [==============================] - 0s 57us/step - loss: 1.8582 - accuracy: 0.3562
Epoch 4/50
800/800 [==============================] - 0s 83us/step - loss: 1.7381 - accuracy: 0.3900
Epoch 5/50
800/800 [==============================] - 0s 84us/step - loss: 1.6293 - accuracy: 0.4250
Epoch 6/50
800/800 [==============================] - 0s 78us/step - loss: 1.5447 - accuracy: 0.4387
Epoch 7/50
800/800 [==============================] - 0s 67us/step - loss: 1.4948 - accuracy: 0.4750
Epoch 8/50
800/800 [==============================] - 0s 80us/step - loss: 1.4412 - accuracy: 0.4900
Epoch 9/50
800/800 [==============================] - 0s 78us/step - loss: 1.3488 - accuracy: 0.5300
Epoch 10/50
800/800 [==============================] - 0s 54us/step - loss: 1.3170 - accur

使用`evaluate`方法进行评估。

In [20]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

200/200 [==============================] - 0s 316us/step
test_acc:  0.6349999904632568
